# Importing Libraries

In [40]:
import numpy as np
import torch
from torch.autograd.functional import jacobian         # computation graph
from torch import Tensor, nn, optim                 # tensor node in the computation graph
# import torch.nn as nn                     # neural networks
# import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
import time
from scipy.integrate import simps
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(0)

# Random number generators in other libraries
np.random.seed(0)

# Loading the datasets from experiments

In [41]:
base_dir = 'exp_data'

# List of all the folders in the base directory
folders = [f for f in glob.glob(os.path.join(base_dir, '*')) if os.path.isdir(f)]

datasets = {}

for folder in folders:
    # List of all the files in the folder
    files = glob.glob(os.path.join(folder, '*.csv'))

    data = []

    for file in files:
        data.append(pd.read_csv(file))
    
    datasets[folder] = data

In [42]:
# HELPER Functions for course project

def computeJacobian(F):
    """
    Compute Jacobian from deformation gradient.
    
    _Input Arguments_
    
    - `F` - deformation gradient in Voigt notation
    
    _Output Arguments_
    
    - `J` - Jacobian
    
    ---
    
    """
    F11 = F[:,0:1]
    F12 = F[:,1:2]
    F21 = F[:,2:3]
    F22 = F[:,3:4]

    J = F11*F22 - F12*F21
    return J

def computeCauchyGreenStrain(F):
    """
    Compute right Cauchy-Green strain tensor from deformation gradient.
    
    _Input Arguments_
    
    - `F` - deformation gradient in Voigt notation
    
    _Output Arguments_
    
    - `C` - Cauchy-Green strain tensor in Voigt notation
    
    ---
    
    """
    F11 = F[:,0:1]
    F12 = F[:,1:2]
    F21 = F[:,2:3]
    F22 = F[:,3:4]

    C11 = F11**2 + F21**2
    C12 = F11*F12 + F21*F22
    C21 = F11*F12 + F21*F22
    C22 = F12**2 + F22**2

    C = torch.cat((C11,C12,C21,C22),dim=1)
    return C


def computeStrainInvariants(C):
    """
    Compute invariants of the Cauchy-Green strain tensor.
    Plane strain is assumed.
    
    _Input Arguments_
    
    - `C` - Cauchy-Green strain tensor in Voigt notation
    
    _Output Arguments_
    
    - `I1` - 1st invariant
    
    - `I2` - 2nd invariant

    - `I3` - 3rd invariant

    ---
    
    """
    C11 = C[:,0:1]
    C12 = C[:,1:2]
    C21 = C[:,2:3]
    C22 = C[:,3:4]

    I1 = C11 + C22 + 1.0
    I2 = C11 + C22 - C12*C21 + C11*C22
    I3 = C11*C22 - C12*C21
    return I1, I2, I3


def computeStrainInvariantDerivatives(F,i,secondDerivative=False):
    """
    Compute derivatives of the invariants of the Cauchy-Green strain tensor with respect to the deformation gradient.
    Plane strain is assumed.
    
    _Input Arguments_
    
    - `F` - deformation gradient in Voigt notation

    - `i` - specify the invariant that should be differentiated 

    - `secondDerivative` - specify if second derivative should be computed 
    
    _Output Arguments_
    
    - `dIdF` - derivative (note that the size of `dIdF` depends on the choice of `secondDerivative`)
    
    ---
    
    """
    F11 = F[:,0:1]
    F12 = F[:,1:2]
    F21 = F[:,2:3]
    F22 = F[:,3:4]
    if not(secondDerivative):
        dIdF = torch.zeros(F.shape[0],F.shape[1])
        if(i==1):
            # dI1/dF:
            dIdF = 2.0*F
        elif(i==2):
            # dI2/dF:
            dIdF11 = 2.0*F11 - 2.0*F12*F21*F22 + 2.0*F11*(F22**2)
            dIdF12 = 2.0*F12 + 2.0*F12*(F21**2) - 2.0*F11*F21*F22
            dIdF21 = 2.0*F21 + 2.0*(F12**2)*F21 - 2.0*F11*F12*F22
            dIdF22 = 2.0*F22 - 2.0*F11*F12*F21 + 2.0*(F11**2)*F22
            dIdF = torch.cat((dIdF11,dIdF12,dIdF21,dIdF22),dim=1)
        elif(i==3):
            # dI3/dF:
            J = F11*F22 - F12*F21
            dIdF11 = 2.0*F22 * J
            dIdF12 = -2.0*F21 * J
            dIdF21 = -2.0*F12 * J
            dIdF22 = 2.0*F11 * J
            dIdF = torch.cat((dIdF11,dIdF12,dIdF21,dIdF22),dim=1)
        else:
            raise ValueError('Incorrect invariant index')
    if secondDerivative:
        dIdF = torch.zeros(F.shape[1],F.shape[1])
        if(i==1):
            # d(dI1/dF)/dF:
            dIdF = 2.0*torch.eye(F.shape[1])
        elif(i==3):
            # d(dI3/dF)/dF:
            J = F11*F22 - F12*F21
            dJdF11 = F22
            dJdF12 = - F21
            dJdF21 = - F12
            dJdF22 = F11
            # d(dI3/dF)/dF11:
            dIdF[0,0] = 2.0 * F22 * dJdF11
            dIdF[0,1] = -2.0 * F21 * dJdF11
            dIdF[0,2] = -2.0 * F12 * dJdF11
            dIdF[0,3] = 2.0 * J + 2.0 * F11 * dJdF11
            # d(dI3/dF)/dF12:
            dIdF[1,0] = 2.0 * F22 * dJdF12
            dIdF[1,1] = -2.0 * F21 * dJdF12
            dIdF[1,2] = -2.0 * J -2.0 * F12 * dJdF12
            dIdF[1,3] = 2.0 * F11 * dJdF12
            # d(dI3/dF)/dF21:
            dIdF[2,0] = 2.0 * F22 * dJdF21
            dIdF[2,1] = -2.0 * J + -2.0 * F21 * dJdF21
            dIdF[2,2] = -2.0 * F12 * dJdF21
            dIdF[2,3] = 2.0 * F11 * dJdF21
            # d(dI3/dF)/dF22:
            dIdF[3,0] = 2.0 * J + 2.0 * F22 * dJdF22
            dIdF[3,1] = -2.0 * F21 * dJdF22
            dIdF[3,2] = -2.0 * F12 * dJdF22
            dIdF[3,3] = 2.0 * F11 * dJdF22
        else:
            raise ValueError('Incorrect invariant index')
    return dIdF    

# def computeFeatures(I1, I2, I3):
def computeFeatures(invariants):
    """
    Compute the features dependent on the right Cauchy-Green strain invariants.
    Note that the features only depend on I1 and I3.
    
    _Input Arguments_
    
    - `I1` - 1st invariant
    
    - `I2` - 2nd invariant

    - `I3` - 3rd invariant
    
    _Output Arguments_
    
    - `x` - features
    
    ---
    
    """
    
    # print('Call1')
    I1, I2, I3 = invariants[:, 0], invariants[:, 1], invariants[:, 2] 
    #Generalized Mooney-Rivlin.
    #The Gent-Thomas model cannot be represented by the generalized
    #Mooney-Rivlin model. An additional feature has to be added.
    considerGentThomas = True
    #Polynomial terms degree.
    Na = 7
    #Volumetric terms degree.
    Nb = 7

    # print('Call2')
    K1 = I1 * torch.pow(I3,-1/3) - 3.0
    K2 = (I1 + I3 - 1) * torch.pow(I3,-2/3) - 3.0
    J = torch.sqrt(I3)
    #Calculate the number of features.
    numFeatures = 0
    #Polynomial terms (dependent on K1 and K2).
    # print('Call3')
    for n in range(Na):
        numFeatures += n + 2
    #Volumetric terms (dependent on J).
    # print('Call4')
    for m in range(Nb):
        numFeatures += 1
    #Additional Gent-Thomas feature.
    # print('Call5')
    if considerGentThomas:
        numFeatures += 1
    #Calculate the features.
    x = torch.zeros(I1.shape[0],numFeatures)
    i =- 1
    
    # print('Call6')
    #Polynomial terms (dependent on K1 and K2).
    for p in range(1,Na+1):
        for q in range(p+1):
            i+=1; x[:,i:(i+1)] = K1**(p-q) * K2**q

    #Volumetric terms (dependent on J):
    # print('Call7')
    for m in range(1,Nb+1):
        i+=1; x[:,i:(i+1)] = (J-1)**(2*m)
        
    #Additional Gent-Thomas feature.
    # print('Call8')
    if considerGentThomas:
        i+=1; x[:,i:(i+1)] = torch.log((K2+3.0)/3.0)

    # print('Call9')
    
    return x

def getNumberOfFeatures():
    """
    Compute number of features.
    
    _Input Arguments_
    
    - _none_
    
    _Output Arguments_
    
    - `features.shape[1]` - number of features
    
    ---
    
    """
    features = computeFeatures(torch.zeros(1,3))
    return features.shape[1]


In [43]:
delta = np.linspace(10,60,6)

for i in range(0,1):
    
    data = datasets[folders[i]][0]
    
    reaction = datasets[folders[i]][1]    
    reaction = torch.tensor(reaction.values, dtype=torch.float32, requires_grad=True)
    
    Xe = data[(data['x_coor'] == 1.0)]
    Xe = Xe.sort_values(by=['y_coor'])
    Xw = data[(data['x_coor'] == 0.0)]
    Xw = Xw.sort_values(by=['y_coor'])
    Xn = data[(data['y_coor'] == 1.0)]
    Xn = Xn.sort_values(by=['x_coor'])
    Xs = data[(data['y_coor'] == 0.0)]
    Xs = Xs.sort_values(by=['x_coor'])
    X_e = Xe[['x_coor', 'y_coor']]
    u_e = Xe[['u_x', 'u_y']]
    X_w = Xw[['x_coor', 'y_coor']]
    u_w = Xw[['u_x', 'u_y']]
    X_n = Xn[['x_coor', 'y_coor']]
    u_n = Xn[['u_x', 'u_y']]
    X_s = Xs[['x_coor', 'y_coor']]
    u_s = Xs[['u_x', 'u_y']]
    
    X_e = torch.tensor(X_e.values, dtype=torch.float32, requires_grad=True)
    u_e = torch.tensor(u_e.values, dtype=torch.float32, requires_grad=True)
    X_w = torch.tensor(X_w.values, dtype=torch.float32, requires_grad=True)
    u_w = torch.tensor(u_w.values, dtype=torch.float32, requires_grad=True)
    X_n = torch.tensor(X_n.values, dtype=torch.float32, requires_grad=True)
    u_n = torch.tensor(u_n.values, dtype=torch.float32, requires_grad=True)
    X_s = torch.tensor(X_s.values, dtype=torch.float32, requires_grad=True)
    u_s = torch.tensor(u_s.values, dtype=torch.float32, requires_grad=True)
    
    
    Xint = data[(data['x_coor'] != 0.0) & (data['x_coor'] != 1.0) & (data['y_coor'] != 0.0) & (data['y_coor'] != 1.0)]
    
    X_int = Xint[['x_coor', 'y_coor']]
    u_int = Xint[['u_x', 'u_y']]
    X_int = torch.tensor(X_int.values, dtype=torch.float32, requires_grad=True)
    u_int = torch.tensor(u_int.values, dtype=torch.float32, requires_grad=True)
    
    batch_size = 62
    num_train_samples = X_int.shape[0]//batch_size 
    
    new_shape = (num_train_samples, batch_size, 2)

    X_int = X_int.reshape(new_shape)
    u_int = u_int.reshape(new_shape)
    
    X_bound = torch.stack((X_s, X_n, X_w, X_e), dim=0)
    u_bound = torch.stack((u_s, u_n, u_w, u_e), dim=0)

In [44]:
# X_bound.shape, u_bound.shape, X_int.shape, u_int.shape

In [45]:
class Physics_Informed_NN(nn.Module):
    
    def __init__(self, layers, num_features, hyperparams=[0.1, 0.01, 0.001]):
        
        super(Physics_Informed_NN, self).__init__()
        
        self._activation = nn.Tanh()
        self._layers = layers
        self._num_layers = len(layers)
        self._loss_function = nn.MSELoss(reduction ='mean')
        self._hyperparams = hyperparams # [0.1, 0.1, 0.1]
        self._num_features = num_features
        
        self._beta = nn.Parameter(torch.zeros((num_features, 1), requires_grad=True))
        
        self._linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        # print(self._linears[0].weight)
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self._linears[i].weight)
            nn.init.ones_(self._linears[i].bias)
            
    def forward(self, x):
        
        for layer in self._linears[:-1]:
            x = self._activation(layer(x))
        x = self._linears[-1](x)
        return x
    
    def loss_criterion(self, u_train, r_train, omega_train, loc=5):
        
        with torch.autograd.enable_grad():
            # Deformation Tensor
            u_train_hat = self.forward(omega_train)
            grad_u_train = jacobian(self.forward, omega_train, create_graph=True)
            
            grad_u_train = torch.stack([grad_u_train[i, :, i, :] for i in range(grad_u_train.shape[0])])
            I = torch.eye(2).unsqueeze(0).repeat(grad_u_train.shape[0], 1, 1)
            F = I + grad_u_train
            F = F.reshape(-1, 4)
            
            J = computeJacobian(F) # Calculate Jacobian
            C = computeCauchyGreenStrain(F) # Calculate Cauchy-Green Strain
            I1, I2, I3 = computeStrainInvariants(C) # Calculate Invariants
            invariants = torch.stack([I1, I2, I3], dim=1)
            invariants.requires_grad_(True)
            
            # Calculate Derivatives of Invariants
            dIdF1 = computeStrainInvariantDerivatives(F, 1)
            dIdF2 = computeStrainInvariantDerivatives(F, 2)
            dIdF3 = computeStrainInvariantDerivatives(F, 3)
            
            Q = computeFeatures(invariants) # Obtain Features
            # st = time.time()
            grad_Q = jacobian(computeFeatures, invariants, create_graph=True) # Obtain Gradient of Features
            # print("Time taken: ", time.time()-st)
            grad_Q = torch.stack([grad_Q[i, :, i, :, 0] for i in range(grad_Q.shape[0])])
            # print(grad_Q)
            # Calculate Derivatives of Features
            dQbdI1 = torch.matmul(grad_Q[:, :, 0], self._beta)
            dQbdI2 = torch.matmul(grad_Q[:, :, 1], self._beta)
            dQbdI3 = torch.matmul(grad_Q[:, :, 2], self._beta)
            # Piola Kirchhoff Stress
            P = dQbdI1 * dIdF1 + dQbdI2 * dIdF2 + dQbdI3 * dIdF3
            
            # grad_P = jacobian(self.eval_Piola, omega_train, create_graph=True)
            
            # u_train_hat, P = self.evaluate_params(omega_train=omega_train)
            # grad_P = jacobian(self.evaluate_params, omega_train, create_graph=True)
            
            # print(P)
            # Pxx, Pxy, Pyx, Pyy = P[:, 0], P[:, 1], P[:, 2], P[:, 3]
            # omega_X, omega_Y = omega_train[:, 0], omega_train[:, 1]
            # dPxxdx = torch.autograd.grad(Pxx, omega_X, grad_outputs=torch.ones(Pxx.shape[0]), create_graph=True, retain_graph=True, allow_unused=True)[0]
            
            # grad_P = jacobian(P, omega_train, create_graph=True)
            
            # print("omega_train: ", omega_train.shape)
            # print("F: ", F.shape)
            # print("J: ", J.shape)
            # print("C: ", C.shape)
            # print("I1: ", I1.shape)
            # print("I2: ", I2.shape)
            # print("I3: ", I3.shape)
            # print("dIdF1: ", dIdF1.shape)
            # print("dIdF2: ", dIdF2.shape)
            # print("dIdF3: ", dIdF3.shape)
            # print("Q: ", Q.shape)
            # print("grad_Q: ", grad_Q.shape)
            # print("dQbdI1: ", dQbdI1.shape)
            # print("dQbdI2: ", dQbdI2.shape)
            # print("dQbdI3: ", dQbdI3.shape)
            # print("P: ", P.shape)
            
            
            # print("grad_P: ", grad_P[0].shape)
            # print("Pxx: ", Pxx.shape)
            # print("Pxy: ", Pxy.shape)
            # print("Pyx: ", Pyx.shape)
            # print("Pyy: ", Pyy.shape)
            # print("omega_X: ", omega_X.shape)
            # print("omega_Y: ", omega_Y.shape)
            # print("dPxxdx: ", dPxxdx.shape)
            
            # Regularization term
            square_params, num = 0.0, 0
            for param in self.parameters():
                num += 1
                square_params += torch.norm(param)**2  # L2 norm of parameters
            # square_weights_sum = 0
            # for layer in self._linears:
            #     square_weights_sum += torch.square(layer.weight).sum()
            # square_beta_sum = self._beta.sum()
            # regularized_params = square_weights_sum + square_beta_sum
                
            # Experimental Loss on Interior Points
            loss_exp = self._loss_function(u_train, u_train_hat)
            
            # Boundary Condition Loss
            loss_bc = torch.tensor(0.0).reshape(1)
            r_train_mod = torch.absolute(r_train)
            # print("r_train_mod: ", r_train_mod)
            if loc == 0: # South Boundary
                x_coord = omega_train[:, 0]
                piola_stress22 = P[:, 3]
                piola_stress12 = P[:, 1]
                resultant22 = (torch.trapz(piola_stress22, x_coord)).reshape(1)
                resultant12 = (torch.trapz(piola_stress12, x_coord)).reshape(1)
                loss_bc += (resultant22-r_train_mod[0])**2 + resultant12**2
                # print("piola_stress: ", piola_stress.shape, x_coord.shape)
                # print("resultant: ", resultant.shape)
                # print("r_train_mod[0]: ", r_train_mod[0].shape)
                # print(loss_bc.shape)
            elif loc == 1: # North Boundary
                x_coord = omega_train[:, 0]
                piola_stress22 = P[:, 3]
                piola_stress12 = P[:, 1]
                resultant22 = (torch.trapz(piola_stress22, x_coord)).reshape(1)
                resultant12 = (torch.trapz(piola_stress12, x_coord)).reshape(1)
                loss_bc += (resultant22-r_train_mod[1])**2 + resultant12**2
            elif loc == 2: # West Boundary
                y_coord = omega_train[:, 1]
                piola_stress11= P[:, 0]
                piola_stress21 = P[:, 2]
                resultant11 = (torch.trapz(piola_stress11, y_coord)).reshape(1)
                resultant21 = (torch.trapz(piola_stress21, y_coord)).reshape(1)
                loss_bc += (resultant11-r_train_mod[2])**2 + resultant21**2
            elif loc == 3:
                y_coord = omega_train[:, 1]
                piola_stress11= P[:, 0]
                piola_stress21 = P[:, 2]
                resultant11 = (torch.trapz(piola_stress11, y_coord)).reshape(1)
                resultant21 = (torch.trapz(piola_stress21, y_coord)).reshape(1)
                loss_bc += (resultant11-r_train_mod[3])**2 + resultant21**2
            
            # PDE Loss
            loss_pde = torch.tensor(0.0) # this loss was turning out to be very small, and the evaluation was very costly
            
            hp1, hp2, hp3 = self._hyperparams
            # total_loss = loss_pde + hp1 * loss_exp + hp2 * loss_bc + hp3 * square_weights_sum
            
            total_loss = loss_pde + hp1 * loss_exp + hp2 * loss_bc + hp3 * square_params
            
        return total_loss

In [46]:
layers = [2,50,10,2]
model = Physics_Informed_NN(layers=layers, num_features=43)
# model.loss_criterion(u_train=u_int[20], r_train=reaction, omega_train=X_int[20])

BATCH = 25
MAX_EPOCHS = 3

# optimizer = optim.LBFGS(model.parameters(), lr=0.1, 
#                               max_iter=5,  
#                               max_eval = None, 
#                               tolerance_grad = 1e-06, 
#                               tolerance_change = 1e-09, 
#                               history_size = 100, 
#                               line_search_fn = 'strong_wolfe')

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

start_time = time.time()

for epoch in range(MAX_EPOCHS):
    print("===============Epoch==========>", epoch)
    # interior_index, boundary_index = 0, 0
    for batch in range(BATCH):
        # print(f'Batch: {batch}')
        if batch <= 3:
            def closure():
                optimizer.zero_grad()
                loss = model.loss_criterion(u_train=u_bound[batch], r_train=reaction, omega_train=X_bound[batch], loc=batch)
                # boundary_index += 1
                loss.backward()
                print(f'Batch: {batch}, Loss: {loss.item()}') # Batch: {batch}, 
                return loss
            optimizer.step(closure)
        else:
            def closure():
                optimizer.zero_grad()
                loss = model.loss_criterion(u_train=u_int[batch - 4], r_train=reaction, omega_train=X_int[batch - 4])
                # interior_index += 1
                print(f'Batch: {batch}, Loss: {loss.item()}')
                loss.backward()
                return loss
            optimizer.step(closure)
            
print('Training Time: ', time.time()-start_time)

===============Epoch==========> 0


Batch: 0, Loss: 0.3346674144268036
Batch: 1, Loss: 0.30927908420562744
Batch: 2, Loss: 0.3156856298446655
Batch: 3, Loss: 0.26928839087486267
Batch: 4, Loss: 0.2694980800151825
Batch: 5, Loss: 0.25597715377807617
Batch: 6, Loss: 0.24758559465408325
Batch: 7, Loss: 0.24115781486034393
Batch: 8, Loss: 0.21475300192832947
Batch: 9, Loss: 0.21606573462486267
Batch: 10, Loss: 0.2014317512512207
Batch: 11, Loss: 0.19223910570144653
Batch: 12, Loss: 0.17780587077140808
Batch: 13, Loss: 0.16942360997200012
Batch: 14, Loss: 0.16500872373580933
Batch: 15, Loss: 0.15341505408287048
Batch: 16, Loss: 0.15160208940505981
Batch: 17, Loss: 0.14301897585391998
Batch: 18, Loss: 0.13862664997577667
Batch: 19, Loss: 0.1289423704147339
Batch: 20, Loss: 0.12543326616287231
Batch: 21, Loss: 0.1183229386806488
Batch: 22, Loss: 0.11405692249536514
Batch: 23, Loss: 0.10914276540279388
Batch: 24, Loss: 0.10815595090389252
===============Epoch==========> 1
Batch: 0, Loss: 0.1090879738330841
Batch: 1, Loss: 0.0994

In [47]:
beta_params = model._beta

# Get the indices for each value
indices = torch.arange(beta_params.size(0))

# Sort the absolute values of beta_params tensor and indices in descending order
sorted_indices = torch.argsort(torch.abs(beta_params.squeeze()), descending=True)
sorted_beta_params = beta_params[sorted_indices]

# Print the corresponding indices and beta values
print("The beta parameters of the model are, ")
for i, beta in zip(sorted_indices, sorted_beta_params):
    print(f"Index: {i.item()+1}, Beta: {beta.item()}")

# for i, beta in enumerate(model._beta):
#     print(f'{i+1} ====> {beta.item()}')
# print(model._beta)

The beta parameters of the model are, 
Index: 36, Beta: -0.028982507064938545
Index: 2, Beta: -0.019829921424388885
Index: 43, Beta: -0.018234340474009514
Index: 1, Beta: -0.01777459867298603
Index: 5, Beta: -0.016955263912677765
Index: 4, Beta: -0.015920501202344894
Index: 3, Beta: -0.014717147685587406
Index: 37, Beta: -0.012779138050973415
Index: 9, Beta: -0.008368752896785736
Index: 8, Beta: -0.007433408405631781
Index: 7, Beta: -0.006407493259757757
Index: 6, Beta: -0.005308269057422876
Index: 20, Beta: 0.0007336471462622285
Index: 19, Beta: 0.0004912199219688773
Index: 16, Beta: -0.00043488582014106214
Index: 17, Beta: -0.00031879753805696964
Index: 15, Beta: -0.00027281633811071515
Index: 14, Beta: -0.000212154453038238
Index: 25, Beta: 0.00018640853522811085
Index: 26, Beta: 0.000160772746312432
Index: 10, Beta: -0.00014509752509184182
Index: 22, Beta: -0.0001206896995427087
Index: 23, Beta: -9.17998404474929e-05
Index: 11, Beta: -6.934937846381217e-05
Index: 24, Beta: 6.765976